In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

import os
import re
import numpy as np
import pandas as pd
import collections

In [2]:
x_name = os.path.join(os.getcwd(), '데이터SET', f"{'[Track1_데이터3] samp_cst_feat'}.csv")
df_x = pd.read_csv(x_name, index_col=0)

y_name = os.path.join(os.getcwd(), '데이터SET', f"{'[Track1_데이터2] samp_train'}.csv")
df_y = pd.read_csv(y_name, index_col=0)

df = pd.merge(df_x, df_y, on='cst_id_di')

In [3]:
df.MRC_ID_DI[df.MRC_ID_DI > 0 ] = 1

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [4]:
nc_name = os.path.join(os.getcwd(), '데이터SET', f"{'[Track1_데이터4] variable_dtype'}.xlsx")
nc = pd.read_excel(nc_name, index_col=0)

In [ ]:
from  sklearn.preprocessing import scale
from statsmodels.nonparametric.kde import KDEUnivariate

def find_outliers_kde(x):
    x_scaled = scale(list(map(float, x)))
    kde = KDEUnivariate(x_scaled)
    kde.fit(bw='scott', fft=True)
    pred = kde.evaluate(x_scaled)
    
    n = sum(pred < 0.01)
    outlier_ind = np.asarray(pred).argsort()[:n]
    outlier_value = np.asarray(x)[outlier_ind]
    
    return outlier_ind, outlier_value

for i in df.columns[0:-1]:
    if nc.loc[i, 'dType'] == 'numerical':
        kde_indices, kde_values = find_outliers_kde(df[i])
        df_50 = df[i].quantile(0.50)
        for j in range(len(df)):
            if df[i].values[j] in kde_values:
                df.at[df.index[j], i] = df_50
"""
for i in df.columns[0:-1]:
    if nc.loc[i, 'dType'] == 'numerical':
        d_90 = df[i].quantile(0.99)
        d_10 = df[i].quantile(0.01)
        d_50 = df[i].quantile(0.50)
        df[i] = np.where(df[i] > d_90, d_90, df[i])
        df[i] = np.where(df[i] < d_10, d_10, df[i])
"""
print(df.shape)
#(10124, 227)

In [ ]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
import random

physical_devices = tf.config.list_physical_devices('GPU') 
tf.config.experimental.set_memory_growth(physical_devices[0], True)

seed_value= 0
os.environ['PYTHONHASHSEED']=str(seed_value)
random.seed(seed_value)
np.random.seed(seed_value)
tf.random.set_seed(seed_value)

In [ ]:
import sys
ipython = get_ipython()

def hide_traceback(exc_tuple=None, filename=None, tb_offset=None,
                      exception_only=False, running_compiled_code=False):
       etype, value, tb = sys.exc_info()
       return ipython._showtraceback(etype, value, ipython.InteractiveTB.get_exception_only(etype, value))

ipython.showtraceback = hide_traceback

In [ ]:
df_0 = df[df['MRC_ID_DI'] == 0].sample(frac=1)
df_1 = df[df['MRC_ID_DI'] == 1].sample(frac=1)

sample_size = len(df_0) if len(df_0) < len(df_1) else len(df_1)

df_h = pd.concat([df_0.head(sample_size), df_1.head(sample_size)]).sample(frac=1)
df_t = pd.concat([df_0.tail(sample_size), df_1.head(sample_size)]).sample(frac=1)
df_f = pd.concat([df_h, df_t]).sample(frac=1)

In [ ]:
X = df_f.drop(columns = ['MRC_ID_DI'], axis=1)
#X = df_f.drop(columns = ['MRC_ID_DI', 'VAR021', 'VAR046'], axis=1)
#y = tf.keras.utils.to_categorical(df_f['MRC_ID_DI'])
y = df_f['MRC_ID_DI']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

In [ ]:
num, cat = list(), list()
for i in X.columns:
    if nc.loc[i, 'dType'] == 'numerical':
        num.append(i)
    else:
        cat.append(i)


X1_train = X_train[num]
X1_test = X_test[num]

X2_train = X_train[cat]
X2_test = X_test[cat]

In [ ]:
from itertools import combinations
from sklearn.preprocessing import PolynomialFeatures

def add_interactions(df1, df2):
    df = pd.concat([df1, df2])
    combos = list(combinations(list(df.columns), 2))
    colnames = list(df.columns) + ['_'.join(x) for x in combos]
    
    poly = PolynomialFeatures(interaction_only=True, include_bias=False)
    df = poly.fit_transform(df)
    df = pd.DataFrame(df)
    df.columns = colnames
    
    noint_indicies = [i for i, x in enumerate(list((df == 0).all())) if x]
    df = df.drop(df.columns[noint_indicies], axis = 1)
    return df[:len(df1)], df[len(df1):]

X1_train, X1_test = add_interactions(X1_train, X1_test)
X2_train, X2_test = add_interactions(X2_train, X2_test)

In [ ]:
"""
import sklearn.feature_selection

select = sklearn.feature_selection.SelectKBest(k=len(X1_train.columns)//16)
selected_features = select.fit(X1_train, y_train)
indices_selected = selected_features.get_support(indices=True)
colnames_selected = [X1_train.columns[i] for i in indices_selected]

X1_train = X1_train[colnames_selected]
X1_test = X1_test[colnames_selected]

select = sklearn.feature_selection.SelectKBest(k=len(X2_train.columns)//16)
selected_features = select.fit(X2_train, y_train)
indices_selected = selected_features.get_support(indices=True)
colnames_selected = [X2_train.columns[i] for i in indices_selected]

X2_train = X2_train[colnames_selected]
X2_test = X2_test[colnames_selected]

"""

In [ ]:

from sklearn.decomposition import PCA
pca = PCA(n_components=min(len(pd.concat([X1_train, X1_test])), len(X1_train.columns)//16))
X_t = pd.DataFrame(pca.fit_transform(pd.concat([X1_train, X1_test])))
X1_train, X1_test = X_t[:len(X1_train)], X_t[len(X1_train):]

pca = PCA(n_components=len(X2_train.columns)//16)
X_t = pd.DataFrame(pca.fit_transform(pd.concat([X2_train, X2_test])))
X2_train, X2_test = X_t[:len(X2_train)], X_t[len(X2_train):]

In [ ]:
y_train = tf.keras.utils.to_categorical(y_train)
y_test = tf.keras.utils.to_categorical(y_test)

In [ ]:
input_1 = tf.keras.Input(dtype = tf.float32, shape = (len(X1_train.columns),))
input_2 = tf.keras.Input(dtype = tf.float32, shape = (len(X2_train.columns),))

dense_layer_1_1 = tf.keras.layers.Dense(units = 64, activation = tf.nn.relu)(input_1)
dropout_1_5 = tf.keras.layers.Dropout(rate = 0.2)(dense_layer_1_1)


dense_layer_2_1 = tf.keras.layers.Dense(units = 10, activation = tf.nn.relu)(input_2)
dropout_2_5 = tf.keras.layers.Dropout(rate = 0.2)(dense_layer_2_1)

concat_layer = tf.keras.layers.Concatenate()([dropout_1_5, dropout_2_5])


dense_layer_3 = tf.keras.layers.Dense(units = 10, activation = tf.nn.relu)(concat_layer)
dropout_3_5 = tf.keras.layers.Dropout(rate = 0.8)(dense_layer_3)

#output = tf.keras.layers.Dense(units = 2, activation = tf.nn.softmax)(dense_layer_3)
output = tf.keras.layers.Dense(units = 2, activation = tf.nn.softmax)(dropout_3_5)

In [ ]:
model = tf.keras.Model(inputs=[input_1, input_2], outputs=output)

In [ ]:
model.compile(loss=tf.keras.losses.CategoricalCrossentropy(), optimizer=tf.compat.v1.train.AdamOptimizer(learning_rate=0.00005), metrics=['acc'])
print(model.summary())

In [ ]:
#history = model.fit(x=X_train, y=y_train, batch_size=32, epochs=400, verbose=1, validation_split=0.1)
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=40)
history = model.fit(x=[X1_train, X2_train], y=y_train, batch_size=32, epochs=2000, verbose=1, validation_split=0.1, callbacks=[callback])

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])

plt.title('model accuracy')
plt.ylabel('acc')
plt.xlabel('epoch')
plt.legend(['train','test'], loc='upper left')
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])

plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train','test'], loc='upper left')
plt.show()

In [ ]:
score = model.evaluate(x=[X1_test, X2_test], y=y_test, verbose=1)

print("Test Score:", score[0])
print("Test ACC:", score[1])